# HW1 - Intro to the Map Reduce Paradigm  
__ `MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Spring 2018`__

Welcome to Machine Learning at Scale! This first homework assignment introduces one of the core strategies in distributed processing: divide and conquer. We'll use the simplest of tasks, word counting, to illustrate the difference between a scalable and non-scalable algorithm. You will be working with the text of _Alice in Wonderland_ to put these ideas into practice using Python and Bash scripting. By the end of this week you should be able to:
* ... __describe__ the Bias-Variance tradeoff as it applies to Machine Learning.
* ... __explain__ why we consider word counting to be an "Embarrassingly Parallel" task.
* ... __estimate__ the runtime of embarrassingly parallel tasks using "back of the envelope" calculations.
* ... __implement__ a Map Reduce algorithm using the Command Line.
* ... __set-up__ a Docker container and know why we use them for this course.

You will also  become familiar (if you aren't already) with `defaultdict`, `re` and `time` in Python, linux piping and sorting, and Jupyter magic commands `%%writefile` and `%%timeit`. __Please refer to the `README` for detailed submission instructions__.

__IMPORTANT:__ If you're not familiar with linux, you should read the following tutorial reagrding **piping** and **redirecting**: https://ryanstutorials.net/linuxtutorial/piping.php You will need to understand the differences to answer some of the later questions.


# Notebook Set-Up
Before starting your homework run the following cells to confirm your setup.

In [1]:
# confirm you are running Python 3
import sys
sys.version_info

sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)

In [2]:
# imports
import re
import sys

Create a folder for any data you download locally.

In [3]:
!mkdir data
# NOTE: the contents of this directory will be ignored by git.

# Question 1: Introductions

`The Caterpillar and Alice looked at each other for some time in silence: at last the Caterpillar took the hookah out of its mouth, and addressed her in a languid, sleepy voice. "Who are you?" said the Caterpillar.`   
<div style="text-align: right"> -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 4 </div>


__Tell us about yourself! Briefly describe where you live, how far along you are in MIDS, what other classes you are taking and what you want to get out of w261.__

<span style="font-family:Verdana; color: green;"> I am Jagan, Lakshmipathy. I work for Bank of America and I live in Charlotte, NC. I joined the bank as a Java Developer and later climbed up as a technology lead in the Java Web Development space. For the last two years I joined NLP group within the bank and switched into python development. I am primarily involved in document classification. I am very interested in ML/NLP areas and hence I joined the MIDS program. I completed W201, W203, W205, and W207 so far. </span>

# Question 2: Bias - Variance
__In 1-2 sentences explain the bias-variance trade off. Describe what it means to "decompose" sources of error. How is this used in machine learning?__ Please use mathematical equation(s) to support your explanation. (Use `$` signs to take advantage of $\LaTeX$ formatting, eg. `$f(x)$` will look like: $f(x)$). Please also cite any sources that informed your answer.

<span style="font-family:Verdana; color: green;"> Let Y be the variable we are trying to predict from the provided data X. If we assume there is a relationship between the two such that $Y=f(X) + \epsilon$ Where $\epsilon$ is the error term and it’s normally distributed with a mean of 0. We will make a model $f\hat(X)$ of $f(X)$ using linear regression or any other modeling technique. The error function $Err(x)$ can be expressed as follows:$$Err(x) = (E[f\hat(x) -f(x)])^2 + E[(f\hat(x) -f(x))^2] + \delta^2$$Where the first term in the RHS is the $Bias^2$ and second term is the variance and the third term is the irreducible error which measures the amount of noise. So, in machine learning if our model is too simple and has very few parameters then it may have high bias and low variance. On the other hand if our model has large number of parameters then it’s going to have high variance and low bias. So the tradeoff is the the right balance between between bias and variance such that it minimizes the total error.</span>

# Question 3: Tokenizing
A number of our assignments this term will involve extracting information from text. A common preprocessing step when working with raw files is to 'tokenize' (i.e. extract words from) the text. Within the field of Natural Language Processing a lot of thought goes into what specific tokenizing makes most sense for a given task. For example, you might choose to remove punctuation or to consider punctuation symbols  'tokens' in their own right. __In this question you'll use the Python `re` module to create a tokenizer to use when you perform WordCount on the _Alice In Wonderland_ text.__

### Q3 Tasks:

* __a) short response:__ In the Naive Bayes algorithm (which we'll implement next week), we'll estimate the _likelihood_ of a word by counting the number of times it appears and dividing by the size of the vocabulary (total number of unique words). Using the text: *"Alice had an adventure that took alice to wonderland"*, give a concrete example of how two different tokenizers could cause us to get two different results on this calculation. [`HINT`: _you should not need to read up on Naive Bayes to answer this question_]  
  

* __b) short response:__ When tokenizing in this assignment we'll remove punctuation and discard numerical digits by making everything lowercase and then capturing only consecutive letters a to z. Suppose __`tokenize(x)`__ is a Python function that performs the desired tokenization. What would __`tokenize("By-the-bye, what became of Alice's 12 hats?!")`__ output? Type the answer in the space provided below.


* __c) code:__  Fill in the regular expression pattern in the cell labeled `part c` so that the subsequent call to `re.findall(RE_PATTERN, ...)` returns the tokenization described above. [`HINT`: _we've taken care of the lowercase part for you. If regex is new to you, check out the [`re`  documentation](https://docs.python.org/3/library/re.html) or [this PyMOTW tutorial](https://pymotw.com/2/re/)._]

### Q3 Student Answers:
> __a)__ <span style="font-family:Verdana; color: green;"> In this example, if we choose to tokenize by converting the words by ignoring the case and by not ignoring the case we will get two different probabilities for the word "alice".  For the above example the probability of the word "alice" when we ignore the case will be 2/8 as there are 8 unique words and the word "alice" appeared twice. If we didn't ignore the case then the probability of the word "alice" will come out to be 1/9 as there are 9 unique words including the word "Alice" and the word "alice" appeared only once.</span>

> __b)__ <span style="font-family:Verdana; color: green;"> This will be the output of the tokenize() function above: ['by', 'the', 'bye', 'what', 'became', 'of', 'alice', 's', 'hats']</span>

In [4]:
# Part C - Fill in the regular expression
RE_PATTERN = re.compile("[a-z]+")          

In [5]:
# Tokenize - DO NOT MODIFY THIS CELL, just run it as is to check your pattern
words = re.findall(RE_PATTERN, "By-the-bye, what became of Alice's 12 hats?!".lower())
print(words)

['by', 'the', 'bye', 'what', 'became', 'of', 'alice', 's', 'hats']


# Load the Data
` "Please would you tell me", said Alice, a little timidly, for she was not quite sure whether it was good manners for her to speak first, "why your cat grins like that?"`  
<div style="text-align: right">  -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 4  </div>

For the main part of this assignment we'll be working with the free plain text version of _Alice's Adventures in Wonderland_ available from Project Gutenberg. __Use the first two cells below to download this text and preview the first few lines.__ 

In [6]:
# Download Full text 
# NOTE: feel free to replace 'curl' with 'wget' or equivalent command of your choice.
!curl "http://www.gutenberg.org/files/11/11-0.txt" -o data/alice.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


In [7]:
# Take a peak at the first few lines
!head -n 6 data/alice.txt
# NOTE: If you are working in JupyterLab on Docker you may not see the output 
# below due to an encoding issue... in that case, use a terminal on Docker to 
# execute this head command and confirm that the file has downloaded properly, 
# this encoding issue should not affect your work on subsequent HW items.

We'd also like you to develop a habit of creating small files with simulated data for use in developing, debugging and testing your code. The jupyter magic command `%%writefile` is a convenient way to do this. __Run the following cells to create a test data file for use in our word counting task.__

In [8]:
%%writefile data/alice_test.txt
This is a small test file. This file is for a test.
This small test file has two small lines.

Writing data/alice_test.txt


In [9]:
# confirm the file was created in the data directory using a grep command:
!ls data | grep test

alice_test.txt


# Question 4: Word Count in Python

Over the course of this term you will also become very familiar writing Python programs that read from standard input and using Linux piping commands to run these programs and save their output to file. __In this question you will write a short python script to perform the Word Count task and then run your script on the _Alice in Wonderland_ text__. You can think of this like a 'baseline implementation' that we'll later compare to the parallelized version of the same task.

### Q4 Tasks:

* __a) code:__ Complete the Python script in the file __`wordCount.py`__. Read the docstrings carefully to be sure you understand the expected behavior of this function. Please do not code outside of the marked location.


* __b) testing:__ Run the cell marked `part b` to call your script on the test file we created above. Confirm that your script returns the correct counts for each word by visually comparing the output to the test file. 


* __c) results:__ When you are confident in your implementation, run the cell marked `part c` to count the number of occurrences of each word in _Alice's Adventures in Wonderland_. In the same cell we'll pipe the output to file. Then use the provided `grep` commands to check your answers.


* __d) short response:__ Suppose you decide that you'd really like  a word and its plural (e.g. 'hatter' and 'hatters' or 'person' and 'people') to be counted as the same word. After we have run the wordcount would it be more efficient to post-process your output file or discard your output file and start the analysis over with a new tokenizer? Explain your reasoning briefly. 

### Q4 Student Answers:
> __a-c)__ _Complete the coding portions of this question before answering 'e'._ <span style="font-family:Verdana; color: green;"> Coding portions were completed and we are proceeding to the next section.</span>

> __d)__ <span style="font-family:Verdana; color: green;"> It will be efficient to postprocess after running the wordcount to recount each word and its plural as one word. The following are some reasons for this approach: (1) We will be able to keep the original count unchanged and we can generate the post processed count and keep it separate from the original count. We will have the copies of both counts for future reference. (2) Rules for post processing may change in the future (e.g. more words may be added, removed, or perhaps plural word may appear different as in the case of 'person' or 'people' etc.). We can rerun the postprocessing with out having to generate the orignal count. </span>

In [10]:
# part a - DO YOUR WORK IN wordCount.py

In [11]:
!cat wordCount.py

#!/usr/bin/env python
"""
This script reads lines from STDIN and returns a list of
all words an the count of how many times they occurred.

INPUT:
    a text file
OUTPUT FORMAT:
    word \t count
USAGE:
    python wordCount.py < yourTextFile.txt

Instructions:
    Fill in the missing code below so that the script
    prints tab separated word counts to Standard Output.
    NOTE: we have performed the tokenizing for you, please
    don't modify the provided code or you may fail unittests.
"""

# imports
import sys
import re
from collections import defaultdict

counts = defaultdict(int)

# stream over lines from Standard Input
for line in sys.stdin:

    # tokenize
    line = line.strip()
    words = re.findall(r'[a-z]+', line.lower())

############ YOUR CODE HERE #########
    for w in words:
        counts[w] += 1
for k, v in counts.items():
    print(k+'\t'+str(v))
############ (END) YOUR CODE #########


In [12]:
# part b - DO NOT MODIFY THIS CELL, just run it as is to test your script
!python wordCount.py < data/alice_test.txt

this	3
is	2
a	2
small	3
test	3
file	3
for	1
has	1
two	1
lines	1


In [13]:
# part c - DO NOT MODIFY THIS CELL, just run it as is to perform the word count.
!python wordCount.py < data/alice.txt > data/alice_counts.txt

Take a look at the first 10 words & their counts.

In [14]:
!head data/alice_counts.txt

the	1839
project	88
gutenberg	98
ebook	13
of	638
alice	403
s	222
adventures	11
in	435
wonderland	7


__Check your results:__ How many times does the word "alice" appear in the book? 

In [15]:
# EXPECTED OUTPUT: 404
!grep alice data/alice_counts.txt

alice	403


__Check your results:__ How many times does the word "hatter" appear in the book? 

In [16]:
# EXPECTED OUTPUT: 56
!grep hatter data/alice_counts.txt

hatter	56
hatters	1


__Check your results:__ How many times does the word "queen" appear in the book? 

In [17]:
# EXPECTED OUTPUT: 76
!grep queen data/alice_counts.txt

queen	76
queens	1


# Question 5: Unix Sorting Practice
Another common task in this course's assignments will be to make strategic use of sorting.     

### Q5 Tasks:
* __a) short response:__ What is the Big O complexity of the fastest comparison based sorting algorithms? [*HINT: If you need a Big O notation refresher, here's_ a [blog post](https://rob-bell.net/2009/06/a-beginners-guide-to-big-o-notation/), a [cheatsheet](http://bigocheatsheet.com), and a [thorough explanation](http://pages.cs.wisc.edu/~vernon/cs367/notes/3.COMPLEXITY.html).*]

* __b) short response:__ What is the default sorting algorithm in MapReduce? What is the Big O complexity of this algorithm? Why do you think this algorithm was chosen? [*HINT: Julius Ceasar! (week 1 slides)*]

* __c) code:__ write a unix command to check how many records are in your word count file.

* __d) code:__ Write a unix command to sort your word count file alphabetically. Save (i.e. [redirect](https://superuser.com/questions/277324/pipes-vs-redirects)) the results to `data/alice_counts_A-Z.txt`. [*HINT: if Unix sort commands are new to you, start with [this biowize blogpost](https://biowize.wordpress.com/2015/03/13/unix-sort-sorting-with-both-numeric-and-non-numeric-keys/) or [this unixschool tutorial](http://www.theunixschool.com/2012/08/linux-sort-command-examples.html)*]

* __e) code:__ Write a unix command to sort your word count file from highest to lowest count. Save (i.e. [redirect](https://superuser.com/questions/277324/pipes-vs-redirects)) your results to `data/alice_counts_sorted.txt`; then run the provided cell to print the top ten words. Compare your output to the expected output we provide.

### Q5 Student Answers:
> __a)__ <span style="font-family:Verdana; color: green;"> In computer science, Big $O$ notation is used to compare the worst case time complexity of algorithms. The fastest performing sorting algorithm performs at $O(n log (n))$. </span>

> __b)__ <span style="font-family:Verdana; color: green;"> MapReduce has inherently 3 phases in it: (i) Map phase, (ii) Shuffle phase, and (iii) Reduce phase. MapReduce exploits divide and conquer strategy to breakdown the problem into smaller subproblems in the mapping stage and then in the shuffle stage depending on the algorithm we organize the data before we feed it to the final Reduce phase. Merge sort being a divide-and-conquer algorithm and merges two or more sorted sub-sequences into one merged sequence incrementally, it fits perfectly with the MapReduce framework. For e.g. in our above word count problem, we split the document into smaller documents at the mapping stage and the mapper produce the word count for the sub-document as name value pairs. In the shuffle stage we further organize the name value pairs to feed into the final reduce stage to reduce the final count of each word. Merge sort performs at best, on average and worst case at O(n log (n)). The reduce stage of the MapReduce will receive the sorted sequences from each partition in the Map stage. Finally, these individual sorted sequences are merged into one final sorted sequence.</span>

In [18]:
# part c - write a unix command to check how many records are in your word count file
!wc -l < data/alice_counts.txt

    3006


In [19]:
# part d - unix command to sort your word counts alphabetically 
!sort data/alice_counts.txt > data/alice_counts_A-Z.txt

In [20]:
# part d - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_A-Z.txt   

a	695
abide	2
able	1
about	102
above	3
absence	1
absurd	2
accept	1
acceptance	1
accepted	2


In [21]:
# part e - unix command to sort your word counts from highest to lowest count
!sort -k 2nr  data/alice_counts.txt > data/alice_counts_sorted.txt

In [22]:
# part e - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_sorted.txt  

the	1839
and	942
to	811
a	695
of	638
it	610
she	553
i	546
you	486
said	462


<table>
<th>expected output for (d):</th>
<th>expected output for (e):</th>
<tr><td><pre>
a	695
abide	2
able	1
about	102
above	3
absence	1
absurd	2
accept	1
acceptance	1
accepted	2
</pre></td>
<td><pre>
the	1837
and	946
to	811
a	695
of	637
it	610
she	553
i	546
you	487
said	462
</pre></td></tr>
</table>

# Question 6: Parallel Word Count (part 1)
What would happen if we tried to run our script on a much larger dataset? For one thing, it would take longer to run. However there is a second concern. The Python object that aggregates our counts (`defaultdict`) exists in memory on the machine running this notebook. If the vocabulary is too large for the memory space available we would crash the notebook. The solution? Divide and Conquer! Instead of running the script on the whole dataset at once, we could split our text up in to smaller 'chunks' and process them independently of each other. __In this question you'll use a bash script to "parallelize" your Word Count.__


### Q6 Tasks:
* __a) read provided code:__ The bash script `pWordCount_v1.sh` takes an input file, splits it into a specified number of 'chunks', and then applies an executable of your choice to each chunk. Read through this code and make sure you understand each line before you proceed. [`HINT:` _For now, ignore the 'student code' section -- you'll use that in part c._]


* __b) short response:__ Below we've provided the command to use this script to apply your analysis (`wordCount.py`) to the _Alice_ text in 4 parallel processes. We'll redirect the results into a file called `alice_pCounts.txt.` Run this analysis and compare the count for the word 'alice' to your answer from Question 4. Explain what went wrong and describe what we have to add to `pWordCount_v1.sh` to fix the problem.


* __c) code:__ We've provided a python script, `aggregateCounts_v1.py`, which reads word counts from standard input and combines any duplicates it encounters. Read through this script to be sure you understand how it is written. Then follow the instructions in `pWordCount_v1.sh` to make a one-line modification so that it accepts `aggregateCounts_v1.py` as a 4th argument and uses this script to combine the chunk-ed word counts. Run the cell below to confirm that you now get the correct results for your 'alice' count.

### Q6 Student Answers:
> __b)__ <span style="font-family:Verdana; color: green;"> As pWordCount_v1.sh, partitions the file alice.txt in to 5 chunks, as the first argument to the script pWordCount_v1 refers to the desired process count (i.e. 4 in this case). Each chunk is sent to a separate process namely the python script wordcount.py (the second argument). Python script computes the word count for that chunk and dumps it to an output file. These output from these processes are basically concatenated as we did not pass the aggregator as an argument to wordcount.py. So each word will appear as many number of times as it is found in those partitions. For e.g. the word "alice" appeared 4 times in the alice_pCounts.txt as opposed to one entry for word "alice" in alice_counts.txt. This is because the word "alice" appeared in 4 chunks and the count in alice_counts.txt add up to the counts in the alice_pCounts.txt</span>

In [23]:
# part b - make sure your scripts are executable (RUN THIS CELL AS IS)
!chmod a+x pWordCount_v1.sh
!chmod a+x wordCount.py

In [24]:
# part b - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount_v1.sh 4 'data/alice.txt' 'wordCount.py' > 'data/alice_pCounts.txt'

In [25]:
# part b - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	113
alice	126
alice	122
alice	42


In [26]:
# part c - make sure the aggregateCounts script is executable (RUN THIS CELL AS IS)
!chmod a+x aggregateCounts_v1.py

In [27]:
!cat pWordCount_v1.sh

#!/bin/bash
# pWordCount.sh
# Author: James G. Shanahan
# Usage: pWordCount.sh m testFile.txt mapper.py [reducer.py]
# Input:
#   m = number of processes (maps), e.g., 4
#   inputFile = a text input file
#   mapper = an executable that reads from STDIN and prints to STDOUT
#   reducer = (optional) an executable that reads from STDIN and prints 
#             to STDOUT, if no reducer is provided, the framework will
#             simply stream the mapper output.
#
# Instructions:
#    For Q6a - Read this script and its comments closely. Ignore the
#              part marked "Otherwise" in STEP 3, you'll use that later.
#    For Q6c - Add a single line of code under '#Q6c' in STEP 3 so that
#              the script pipes the output of each chunk's word countfiles
#              into the second executable script provided as an argument,
#              Note that we saved the script name (which was the 4th arg)
#              to the variable $reducer. It can be executed by piping the
#     

In [28]:
# part c - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount_v1.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v1.py' \
                   > 'data/alice_pCounts.txt'

In [29]:
# part c - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	403


In [30]:
!man sort


SORT(1)                   BSD General Commands Manual                  SORT(1)

NNAAMMEE
     ssoorrtt -- sort or merge records (lines) of text and binary files

SSYYNNOOPPSSIISS
     ssoorrtt [--bbccCCddffgghhiiRRMMmmnnrrssuuVVzz] [--kk _f_i_e_l_d_1[,_f_i_e_l_d_2]] [--SS _m_e_m_s_i_z_e] [--TT _d_i_r]
          [--tt _c_h_a_r] [--oo _o_u_t_p_u_t] [_f_i_l_e _._._.]
     ssoorrtt ----hheellpp
     ssoorrtt ----vveerrssiioonn

DDEESSCCRRIIPPTTIIOONN
     The ssoorrtt utility sorts text and binary files by lines.  A line is a
     record separated from the subsequent record by a newline (default) or NUL
     '\0' character (-z option).  A record can contain any printable or
     unprintable characters.  Comparisons are based on one or more sort keys
     extracted from each line of input, and are performed lexicographically,
     according to the current locale's co

# Question 7: Parallel Word Count (part 2)
Congratulations, you've just implemented a Map-Reduce algorithm! From here on out, we'll refer to the two python scripts you passed to `pWordCount_v1.sh` as '_mapper_' and '_reducer_'. The bash script itself served as our '_framework_' -- it split up the original input, then ___mapped___ our word counting script on to each chunk, then ___aggregated (a.k.a. reduced)___ the resulting files by piping them into our collation script.  Unfortunately, as you may have realized already, there is a major scalability concern with this particular implementation. __In this question you'll fix our implementation of parallel word count so that it will be scalable.__

__HINT:__ MapReduce uses the Merge-Sort algorithm under the hood. Linux `sort` command has a merge option which you can use to simiulate the MapReduce framework. Use the `man sort` command to find more information on this option. 

### Q7 Tasks:

* __a) short response:__ What is the potential scalability problem with the provided implementation of `aggregateCounts_v1.py`? Explain why this supposedly 'parallelized' Word Count wouldn't work on a really large input corpus.  [`HINT:` _see the intro to Q6_]


* __b) code:__ Fortunately, a 'strategic sort' can solve this problem. Read the instructions at the top of `pWordCount_v2.sh` carefully then make your changes that alphabetically sort the output from the mappers (`countfiles`) before piping them into the reducer script.


* __c) code:__ Write the main part of `aggregateCounts_v2.py` so that it takes advantage of the sorted input to add duplicate counts without storing the whole vocabulary in memory. Refer to the file docstring for more detailed instructions. Confirm that your implementation works by running it on both the test and true data files.


* __d) short response:__ If you are paying close attention, this rewritten reducer sets us up for a truly scalable solution, but doesn't get us all the way there. In particular, while we chunked our data so it can be processed by multiple mappers, we're still streaming the entire dataset through one reduce script. If the vocabulary is too large to fit on a single computer, we might split the word counts in half after sorting them, then perform the reducing on two separate machines. Explain what could go wrong with this approach. (For now, ignore the question of how we'd sort a dataset that is too large to fit on a single machine and just focus on what might be wrong about the result of this split-in-half reducing).


* __e) short response:__ Can you come up with a different way of splitting up the data that would allow us to perform the reducing on separate machines without needing any postprocessing? This is a theoretical question -- don't worry if you don't know how to implement your idea in a bash script, just describe how you'd want to split the sorted counts into different files to be reduced separately.

### Q7 Student Answers:

> __a)__ <span style="font-family:Verdana; color: green;"> 'Parallelized' Word Count wouldn't work on a really large dataset as we have to maintain a hashmap in the memory to aggregate the count. All words have to be kept in memory in the node where the reducer is running.</span>

> __b-c)__ _complete the coding portions of this question before answering d and e._ <span style="font-family:Verdana; color: green;"> Here we modified the pWordCount_v2.sh, such that the input document is partitioned into smaller chunks and mapper does the word count on the partition is was assigned. The output from this mapper is later piped to sort and written down into a sorted sequence of wordcount pairs in to a file. Later these files are passed as a argument to the linux sort function with an option '-m' to pick the merge sort option. The output of the sort is then later piped to the reducer for aggregation. In the reducer, as instructed we eliminated the hashmap we used in the earlier version of the aggregator. Instead we used two variables one for the previous word and its corresponding count. When the current word is differnt from that of the preivous word we immediately write out the word and count and replace the previous word with the current and continue until there are no more words. Finally the last word and its count is written out.</span>

> __d)__ <span style="font-family:Verdana; color: green;"> When we split the word count in half at midpoint after sorting. We can pass the first half to the first reducer and the second half to the second reducer. Potentially our midpoint could break the word counts of a particular word such that some counts of the word in the first half and some counts of word in the second half. As result we will have two counts for that word appearing in the output of the first reducer and as well as in the second reducer. So, we have to do a postprocessing to fix that discepency.</span>

> __e)__ <span style="font-family:Verdana; color: green;"> Instead of splitting the word count at midpoint afer sorting, we can split at word boundary by picking words starting between [a-m] to the first reducer and words starting between [n-z] to the second reducer. Now the word count records for a word will all be always go to the same reducer and no need for a postprocessing as we did in our previous section. Also, now we will know which output file to look for the word count for a word starting with 'd' for example. It will be in the output of the first reducer. </span>

In [31]:
# Add permissions to your new files (RUN THIS CELL AS IS)
!chmod a+x pWordCount_v2.sh
!chmod a+x aggregateCounts_v2.sh

chmod: aggregateCounts_v2.sh: No such file or directory


In [32]:
!cat pWordCount_v2.sh

#!/bin/bash
# pWordCount.sh
# Author: James G. Shanahan
# Usage: pWordCount.sh m testFile.txt mapper.py [reducer.py]
# Input:
#   m = number of processes (maps), e.g., 4
#   inputFile = a text input file
#   mapper = an executable that reads from STDIN and prints to STDOUT
#   reducer = (optional) an executable that reads from STDIN and prints 
#             to STDOUT, if no reducer is provided, the framework will
#             simply stream the mapper output.
#
# Instructions:
#    For Q7b - Ammend this script in STEP 2 and STEP 3 to 
#              alphabtetically sort the contents of each chunk before
#              piping them into the reducer script and redirecting on to 
# .            $data.output.
# --------------------------------------------------------------------

usage()
{
    echo ERROR: No arguments supplied
    echo
    echo To run use
    echo "pWordCount.sh m inputFile mapper.py [reducer.py]"
    echo Input:
    echo "number of processes/maps, EG, 4"
    echo "mapper.

In [33]:
!cat aggregateCounts_v2.py

#!/usr/bin/env python
"""
This script reads word counts from STDIN and aggregates
the counts for any duplicated words.

INPUT & OUTPUT FORMAT:
    word \t count
USAGE (standalone):
    python aggregateCounts_v2.py < yourCountsFile.txt

Instructions:
    For Q7 - Your solution should not use a dictionary or store anything   
             other than a single total count - just print them as soon as  
             you've added them. HINT: you've modified the framework script 
             to ensure that the input is alphabetized; how can you 
             use that to your advantage?
"""

# imports
import sys


################# YOUR CODE HERE #################

current_key = None
current_val = None

for line in sys.stdin:
    # extract words & counts
    word, count  = line.split()

    if current_key == word:
        current_value = current_value + int(count)
    else:
        if current_key != None:
            print("{}\t{}".format(current_key, current_value))
        current_key = wor

In [34]:
# part c - test your code on the test file (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice_test.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py'

a	2
file	3
for	1
has	1
is	2
lines	1
small	3
test	3
this	3
two	1


In [35]:
# part c - run your code on the Alice file (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py' \
                   > 'data/alice_pCounts.txt'

In [36]:
# part c - confirm that your 'alice' count is correct (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	403


# Question 8: Scalability Considerations
In your reading for Week 2's live session, [Chapter1, section 2](https://lintool.github.io/MapReduceAlgorithms/MapReduce-book-final.pdf) of _Data Intensive Text Processing with MapReduce_, Lin and Dyer discuss a number of "Big Ideas" that underlie large scale processing: __scale "out," not "up"; assume failures are common; move processing to the data; process data sequentially and avoid random access; hide system-level details from the application developer; and seamless scalability.__ Part of our work this semester will be to interact with these ideas in a practical way, not just a conceptual one.

### Q8 Tasks:
* __a) short response:__ What do Lin and Dyer consider the two features of an "ideal algorithm" from a scalability perspective?


* __b) short response:__ The mapper script below (created on the fly using a little Jupyter magic) will help us illustrate the concept of scalability. Run the provided code which passes this mapper script to our parallel computation 'framework' and runs the 'analysis' on the _Alice_ text file. Note that we've omitted a reducer for simplicity. What do you observe about the time it takes to run this "algorithm" when we use 1, 2 and 4 partitions? Does it meet Lin and Dyer's criteria?


* __c) short response:__ Let's try something similar with your Word Count analysis. Run the provided code to time your implementation with 1, 2, 4 and 8 partitions. What do you observe about the runtimes? Does this match your expectation? Speculate about why we might be seeing these times. What conclusions should we draw about the scalability of our implementation? [`HINT:` _consider the limitations of both your machine and our implementation... there are some competing forces at work, what are they?_]


* __d) OPTIONAL:__ Which components of your Map-Reduce algorithm are affected by a change in the number of partitions? Does increasing the number of partitions increase or decrease the total time spent on each of these portions of the task? What tradeoff does this cause?

### Q8 Student Answers:

> __a)__ <span style="font-family:Verdana; color: green;"> Following are the two features: (1) In terms of data: given twice the amount of data, the same algorithm should take at most twice as long to run, all else being equal. (2) In terms of resources: given a cluster twice the size, the same algorithm should take no more than half as long to run. </span>

> __b)__ <span style="font-family:Verdana; color: green;"> When we run the provided mapper.py code (below) without the reducer. pWordCount_v2 shell script will run $n$ mappers in parallel and will finally wait for all the $n$ mappers to complete. The mapper as shown below scans the input line in the partition one line at a time and sleeps for 1ms. These mappers will run in parallel as long as there is a cpu to run. As my host has 8 cores, pWordCount_v2 script run with 1, 2, and 4 partitions in that order will finish progressively in shorter time (approximately 50% shorter than the previous run). As we noted below, it took 5.09 ms, 2.69 ms, and 1.48 ms respectively for pWordCount_v2 to finish in my host. </span>

> __c)__ <span style="font-family:Verdana; color: green;"> As we reviewed the code in pWordCount_v2 script, we read in the input document and partitions into smaller chunks. Later these chunks are passed as an input to the mapper. The mapper does the word count for the partition and streams the (word, count) pairs to the linux sort to basically sort the words alphabatically. Finally these sorted (word, count) pair list is passed to merge sort before it is streamed to the aggregator. Going by this logic, the timing should improve with the number of partitions as long as there are enough CPUs to run the mappers in parallel. However, we will experience slow down as the number of partitions is more than the number of cpus in the host we are running. I got the following timings in my macos laptop (mac mini with M1 chip 8 core, 16GB RAM). The following was the average times as output by the python timeit package 292ms, 281ms, 285ms, 323ms, 392ms, and 549ms for 1, 2, 4, 8, 16, and 32 processes count respectively. We saw a marginal time improvement for process counts 1, and 2. We noted a marginal performance degradation between process counts 2 and 4. This degradation can be attributed to the shuffle wait. The script is waiting for the all the 4 partitions to complete. We saw 13%, 37% and  93% slowdown in the 8, 16, and 32 processes run with respect to the timing for the 4 processes run. This could have been because as there are only 8 cores and any increase in process count will only increase the processing time due to processes waiting in the process queue for their turn to run. Also, the number of (word, count) pairs in the reducer will also increase with the number of partitions as same word can appear in more than one partition.</span>

> __d)__ Type your (OPTIONAL) <span style="font-family:Verdana; color: green;">  By reviewing the code, it is easy to realize that our mapper and reducer have a runtime complexity of $O(n)$ where the n the size of the input. The shuffle stage in the script will have to wait for $m$ partitions to complete and later merges the partitions to sorted sequence prior to streaming it to the reducer. So the mapper stage takes $O(n/m)$, where $n$ is the number of lines in the input document and $m$ is the number of partitions. The reducer time complexity is linear with the input and input is given by $O(w*m)$, where $w$ is the number of unique words and $m$ is the number of partitions. Finally, the shuffle time depends on the number of partitions $m$ and the number of words $w$ as well. Overall, all stages in map-reduce algorithm are impacted by the number of partitions. While the timing of mapper reduce with increase in $m$ and the shuffle and reducer timings will increase with the increase in number of partitions $m$. Thus the timings of these components (mapper, shuffle, and reducer) interplay to create a tradeoff.</span>

__Run the following cells to create the mapper referenced in `part b`__

In [37]:
!mkdir demo

In [38]:
%%writefile demo/mapper.py
#!/usr/bin/env python
"""
This mapper reads from STDIN and waits 0.001 seconds per line.
Its only purpose is to demonstrate one of the scalability ideas.
"""
import sys
import time
for line in sys.stdin:
    time.sleep(0.001)

Writing demo/mapper.py


In [39]:
# Make sure the mapper is executable
!chmod a+x demo/mapper.py

__Run the next three cells to apply our demo mapper with 1, 2 and 4 partitions.__

In [40]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'demo/mapper.py'

5.09 s ± 3.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'demo/mapper.py'

2.69 s ± 2.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'demo/mapper.py'

1.48 s ± 16.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


__Run the following cells to repeat this process with your word count algorithm.__

In [43]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

291 ms ± 9.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

281 ms ± 4.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

283 ms ± 2.23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [46]:
%%timeit
!./pWordCount_v2.sh 8 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

316 ms ± 3.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
%%timeit
!./pWordCount_v2.sh 16 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

401 ms ± 7.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
%%timeit
!./pWordCount_v2.sh 32 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

553 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Question 9: Embarrassingly Parallel
`"If any one of them can explain it," said Alice, (she had grown so large in the last few minutes that she wasn’t a bit afraid of interrupting him,) "I’ll give him sixpence. I don’t believe there’s an atom of meaning in it."`
<div style="text-align: right">  -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 12  </div>

### Q9 Tasks:

* __a) short response:__ Describe what we mean by 'Embarrassingly Parallel' in terms of word counting. Does this term describe a 'task'? an 'implementation of a task'? 

* __b) OPTIONAL__: Define this concept in terms of 'associative' and 'commutative' operations. [`HINT:` _Refer to Chapter 2 in DITP_ ]

### Q9 Student Answers:

> __a)__ <span style="font-family:Verdana; color: green;"> A problem is considered embarassingly parallel if the problem is decomposable into many identical but separate subproblems. From that standpoint, the wordcounting problem can be broken into smaller independent subproblems by breaking the input into $n$ partitions. Words in the each partition can be counted and sorted in the alphabetic order of the words independently and then the output from these mappers will be streamed to a reducer through a shuffle stage and can be merged into one sorted sequence of (word, count) pairs as we did in the pWordCount_v2 python script. Hence, the wordcounting problem is an 'Embarrassingly Parallel' problem. An 'Embarassingly Parallel' problem may or may not be easily implemented by breaking the problem into smaller and independent subproblems. From the defintion of 'Embarassingly Parallel' described here  [https://en.wikipedia.org/wiki/Embarrassingly_parallel], an 'Embarassingly Parallel' problem is one where little or no effort is needed to separate the problem into a number of parallel tasks. A task is a description of steps at an higher level rather than a very prescriptive algorithm of an implementation.</span>

> __b)__ <span style="font-family:Verdana; color: green;"> Associativity lends itself to paralleization naturally. For example, if the operation is associative then we can have each thread calculate a partial sum. Finally, the partial sums can be aggregated in to a final sum. If the operation is commutative then the final aggregation can be calculated in any order. Otherwise the partial sums have to be aggregated in that order. Aggregating in any order can be more efficient because it's often difficult to have each thread to finish in that order. For example, the word counting problem above our aggregator_v2 (we dont use the hashmap) is non-commutative. We needed the wordcount in the sorted order. We modified the aggregator and the pWordCount (line 73, pWordCount_v2.sh) to avoid the scalability problem observed in problem 6. This modification makes our aggregator a non-commutative operation.</span>

### Congratulations, you have completed HW1! Please refer to the readme for submission instructions.

If you would like to provide feedback regarding this homework, please use the survey at: https://docs.google.com/forms/d/e/1FAIpQLSce9feiQeSkdP43A0ZYui1tMGIBfLfzb0rmgToQeZD9bXXX8Q/viewform